In [1]:
import numpy as np
import pandas as pd
import lazypredict

In [5]:
test_scores = pd.read_csv("datasets/Admission and Test Scores/ADM_2015-2021_data.csv")
test_scores.head()

,unitid,admcon1,admcon2,admcon3,admcon4,admcon5,admcon6,admcon7,admcon8,admcon9,...,actcm25,actcm75,acten25,acten75,actmt25,actmt75,satwr25,satwr75,actwr25,year
0,100654,Required,Recommended,Required,Recommended,Neither required nor recommended,Recommended,Considered but not required,Required,Neither required nor recommended,...,15.0,20.0,14.0,20.0,15.0,NaN,NaN,NaN,NaN,2021
1,100663,Required,Neither required nor recommended,Required,Required,Neither required nor recommended,Neither required nor recommended,Considered but not required,Neither required nor recommended,Neither required nor recommended,...,23.0,30.0,23.0,33.0,21.0,NaN,NaN,NaN,NaN,2021
2,100706,Required,Recommended,Required,Neither required nor recommended,Neither required nor recommended,Recommended,Required,Required,Neither required nor recommended,...,24.0,31.0,24.0,32.0,23.0,NaN,NaN,NaN,NaN,2021
3,100724,Required,Neither required nor recommended,Recommended,Neither required nor recommended,Neither required nor recommended,Neither required nor recommended,Considered but not required,Required,Neither required nor recommended,...,15.0,20.0,12.0,22.0,15.0,NaN,NaN,NaN,NaN,2021
4,100751,Required,Recommended,Required,Required,Neither required nor recommended,Neither required nor recommended,Considered but not required,Required,Neither required nor recommended,...,21.0,31.0,21.0,33.0,19.0,NaN,NaN,NaN,NaN,2021
